<a href="https://colab.research.google.com/github/FreshWaterLee/Personal_Stduy_Deep/blob/main/Moblienet_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/tensorflow/models

In [ ]:
from __future__ import print_function
from IPython import display 
base_name = checkpoint_name = 'mobilenet_v2_1.0_224' #@param
url = 'https://storage.googleapis.com/mobilenet_v2/checkpoints/' + checkpoint_name + '.tgz'
print('Downloading from ', url)
!wget {url}
print('Unpacking')
!tar -xvf {checkpoint_name}.tgz
checkpoint = checkpoint_name + '.ckpt'

display.clear_output()
print('Successfully downloaded checkpoint from ', url,
      '. It is available as', checkpoint)


In [ ]:
!wget https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG -O panda.jpg

In [ ]:
import sys
sys.path.append('/content/models/research/slim')

In [ ]:
!pip install tf_slim 

In [ ]:
import tensorflow.compat.v1 as tf
from nets.mobilenet import mobilenet_v2
import tf_slim

tf.compat.v1.disable_eager_execution()

tf.reset_default_graph()
file_input = tf.placeholder(tf.string,())
#placeholder(tf.string, ())

image = tf.image.decode_jpeg(tf.read_file(file_input))

images = tf.expand_dims(image, 0)
images = tf.cast(images, tf.float32) / 128.  - 1
images.set_shape((None, None, None, 3))
images = tf.image.resize_images(images, (224, 224))

# Note: arg_scope is optional for inference.

with tf_slim.arg_scope(mobilenet_v2.training_scope(is_training=False)):
  logits, endpoints = mobilenet_v2.mobilenet(images)
  
# Restore using exponential moving average since it produces (1.5-2%) higher 
# accuracy
ema = tf.train.ExponentialMovingAverage(0.999)
vars = ema.variables_to_restore()

saver = tf.train.Saver(vars)  ## 모델 저장 함수

In [ ]:
from IPython import display
import pylab
from datasets import imagenet
import PIL
display.display(display.Image('/content/panda.jpg'))

with tf.Session() as sess:
  saver.restore(sess,  checkpoint)
  x = endpoints['Predictions'].eval(feed_dict={file_input: '/content/panda.jpg'})
label_map = imagenet.create_readable_names_for_imagenet_labels()  
print("Top 1 prediction: ", x.argmax(),label_map[x.argmax()], x.max())


In [ ]:
import numpy as np
img = np.array(PIL.Image.open('/content/panda.jpg').resize((224, 224))).astype(np.float) / 128 - 1
## 검출할 이미지와 같은 픽셀 배열 생성
gd = tf.GraphDef.FromString(open(checkpoint_name + '_frozen.pb', 'rb').read()) 
## 그래프(점과 선으로 구성된 자료구조)데이터 불러오기
inp, predictions = tf.import_graph_def(gd,  return_elements = ['input:0', 'MobilenetV2/Predictions/Reshape_1:0'])

In [ ]:
## 많은 메모리 할당이 필요하므로 사용후 바로 할당한 메모리를 반납하기 위해 with문을 사용
with tf.Session(graph=inp.graph):
    x = predictions.eval(feed_dict={inp: img.reshape(1, 224,224, 3)})

label_map = imagenet.create_readable_names_for_imagenet_labels()
print("Top 1 Prediction: ", x.argmax(),label_map[x.argmax()], x.max())